In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# 0. Предварительное знакомство с данными.

**0.1. Постановка задачи**

Работа в данном проекте предполагает работу с датасетами, в которых содержатся сведения об отзывах на отели Европы. Модель, которую необходимо  будет обучить, должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасетах данных. 

**0.2. Сведения о датасетах**

Первоначальная версия датасета содержит 17 полей со следующей информацией:
hotel_address — адрес отеля;
review_date — дата, когда рецензент разместил соответствующий отзыв;
average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
hotel_name — название отеля;
reviewer_nationality — страна рецензента;
negative_review — отрицательный отзыв, который рецензент дал отелю;
review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
positive_review — положительный отзыв, который рецензент дал отелю;
review_total_positive_word_counts — общее количество слов в положительном отзыве.
reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
total_number_of_reviews — общее количество действительных отзывов об отеле;
tags — теги, которые рецензент дал отелю;
days_since_review — количество дней между датой проверки и датой очистки;
additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
lat — географическая широта отеля;
lng — географическая долгота отеля.

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
# Ознакомимся с датасетами до проведения всех преобразований
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

#data = df_test.concat(df_train, sort=False).reset_index(drop=True) # объединяем
data = pd.concat([df_train, df_test])

In [ ]:
# Проверим результаты объединения датасетов
data.info()

**Вывод по разделу:**

Датасет "df_train" содержит всего 17 признаков(в том числе признак "reviewer_score") и 386803 строк.

Датасет "df_test" содержит всего 16 признаков и 128935 строк.

Объединеный датасет "data" содержит 18 признаков (добавился признак "sample") и 515738 строк. Пропуски имеют признаки "lat " и "lng".


# 1. Очистка данных от пропущенных значений. 


In [ ]:
# Выделим из всего датасета "data" наименования отелей, где есть пропуски координат.
data[data['lat'].isnull()]['hotel_name'].value_counts()

In [ ]:
# Составим словарь наименований отеля и их координат
coord_dict = {
    'Fleming s Selection Hotel Wien City':[48.209095, 16.354568],
    'Hotel City Central':[48.213560, 16.379923],
    'Hotel Atlanta':[48.220310, 16.355880],
    'Maison Albar Hotel Paris Op ra Diamond':[48.875140, 2.323420],
    'Hotel Daniel Vienna':[48.188835, 16.383810],
    'Hotel Pension Baron am Schottentor':[48.216705, 16.359820],
    'Austria Trend Hotel Schloss Wilhelminenberg Wien':[48.219555, 16.285566],
    'NH Collection Barcelona Podium':[41.391430, 2.177890],
    'Derag Livinghotel Kaiser Franz Joseph Vienna':[48.245914, 16.341188],
    'City Hotel Deutschmeister':[48.220856, 16.366642],
    'Holiday Inn Paris Montmartre':[48.888860, 2.333190],
    'Hotel Park Villa':[48.233495, 16.345556],
    'Cordial Theaterhotel Wien':[48.209530, 16.351515],
    'Roomz Vienna':[48.22201, 16.39331],
    'Mercure Paris Gare Montparnasse':[48.839701, 2.323519],
    'Hotel Advance':[41.38322, 2.16295],
    'Renaissance Barcelona Hotel':[41.392430, 2.167500]
}

In [ ]:
# Заполним пропуски в датасете
def fill_coords(row):
    if pd.isna(row.lat):
        coord = coord_dict[row.hotel_name]
        if coord is not None:
            row.lat = coord[0]
            row.lng = coord[1]
            return row
    else:
        return row

data = data.apply(lambda row: fill_coords(row), axis=1)

In [ ]:
# Проверим результаты преобразований
data.info()

**Вывод по разделу 1:**

В результате проведенных преобразований были выявлены пропуски в признаках "Lat" и "Lng" с последующей заменой данных пропусков актуальными координатами.

# **2. Создание новых признаков и их кодирование.**


2.1 Получение название страны из признака "hotel_address". Создание нового признака и кодирование.

In [ ]:
# Создаем пустой столбец для города
data['city'] = ''

# Функция для извлечения города из адреса
def extract_city(hotel_address):
    parts = hotel_address.split(' ')
    # Предполагаем, что город - последний элемент перед страной
    city = parts[-2].strip()  
    return city

# Применяем функцию к каждому адресу
data['city'] = data['hotel_address'].apply(extract_city)

In [ ]:
# Закодируем признак "city"
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['city_encoded'] = label_encoder.fit_transform(data['city'])

In [ ]:
# Проверим результат преобразований
data.info()

In [ ]:
data.head(10)


2.2 Анализ даты оставления отзыва. Создание новых признаков на основе даты и кодирование.

In [ ]:
# Перед проведением преобразований проверим датасет
data.info()

In [ ]:
#Из признака 'review_date' выделим номер месяца
# Создаем пустой столбец для месяца
data['month'] = ''

# Функция для извлечения месяца из даты
def extract_month(review_date):
    parts_1 = review_date.split('/')
    # Месяц второй элемент в дате
    month = parts_1[0].strip()  
    return month

# Применяем функцию к каждому месяцу
data['month'] = data['review_date'].apply(extract_month)

In [ ]:
# Далее проведем кодирование признака "month" с помощью Label Encoding:

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['month_encoded'] = label_encoder.fit_transform(data['month'])

# Затем создадим новый признак "season" на основе значения "month_encoded":
def get_season(month):
    if month in [11, 0, 1]:
        return 'winter'
    elif month in [2, 3, 4]:
        return 'spring'
    elif month in [5, 6, 7]:
        return 'summer'
    else:
        return 'autumn'

data['season'] = data['month_encoded'].apply(get_season)

In [ ]:
# Для кодирования признака "season" можно воспользоваться методом 
# One-Hot Encoding, который создаст отдельные бинарные столбцы для каждого 
# уникального значения признака "season"
data = pd.get_dummies(data, columns=['season'], prefix='season', drop_first=True)

In [ ]:
# Проверим результат преобразований
data.head(2)

In [ ]:
data.info()

2.3 Анализ тегов (tags) и создание новых признаков.

In [ ]:
# Проанализируем теги и содадим новый признак 'tag_stayed'

In [ ]:
import numpy as np

In [ ]:
# Выделим из тегов количество ночей пребывания гостей в отеле
def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

data['tags_n'] = data['tags'].apply(rev_func)

In [ ]:
data['tags_n']

In [ ]:
# Создадим новый признак "tag_stayed"
def find_stayed(row):
    
    for tag in row:
        if 'Stayed' in tag:
            return tag.split()[1]
    return None

data['tag_stayed'] = data['tags_n'].apply(find_stayed)

In [ ]:
data['tag_stayed']

In [ ]:
# Проведем кодировку признака "tag_stayed"
import category_encoders as ce

binary_encoder = ce.BinaryEncoder(cols=['tag_stayed'])
data = binary_encoder.fit_transform(data)

In [ ]:
# Проверим результаты преобразований
data.info()

In [ ]:
# Создадим новый признак 'trip_type'
# Из признака 'tags_n' выделим тип путешествия

def find_trip_type(row):
    
    for tag in row:
        if 'Solo' in tag:
            return tag.split()[0]
    return 'Couple'

data['trip_type'] = data['tags_n'].apply(find_trip_type)

# Применяем One-Hot Encoding к признаку 'trip_type'
data = pd.get_dummies(data, columns=['trip_type'])

In [ ]:
# Проверим результат проведенных преобразований
data.head(3)


2.4 Кодирование признака "reviewer_nationality"

In [ ]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder(cols=['reviewer_nationality'])
data = binary_encoder.fit_transform(data)

In [ ]:
# Проверим результаты проведенных преобразований 
data.info()

**Вывод по разделу 2:**

В данном разделе были проведены преобразования, позволяющие выделять из признаков типа "object" необходимую информацию. Из признака "review_date " был выделен номер месяца проживания, что далее позволило получить новые признаки, характеризующие сезонность проживания. Был проанализирован признак "tags". Это позволило получить новые признаки, характеризующие количество проведенных гостями ночей и тип проживания - семейный или одиночный. Соответственно была проведена кодировка вновь полученных признаков типа "object".




# 3. Анализ текстовых отзывов.

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('vader_lexicon')
# Создание экземпляра SentimentIntensityAnalyzer
sent_analyzer = SentimentIntensityAnalyzer()

# Функция для анализа тональности текста
def analyze_sentiment(text):
    return sent_analyzer.polarity_scores(text)['compound']

# Применение функции к признаку "negative_review" в датасете
data['negative_review_sentiment'] = data['negative_review'].apply(analyze_sentiment)
data['positive_review_sentiment'] = data['positive_review'].apply(analyze_sentiment)

In [ ]:
# Проверим результат проведенных преобразований.
data.head(2)

In [ ]:
data.info()

**Проведем подготовку датасета для дальнейшего анализа**

In [ ]:

# Удаление признаков типа object
data = data.select_dtypes(exclude=['object'])
data.info()

In [ ]:
#Разделим признаки на категориальные и числовые
# Категориальные признаки
categorical_features = ['reviewer_nationality_0', 'reviewer_nationality_1', 'reviewer_nationality_2', 'reviewer_nationality_3',
                        'reviewer_nationality_4', 'reviewer_nationality_5', 'reviewer_nationality_6', 'reviewer_nationality_7',
                        'city_encoded', 'month_encoded', 'season_spring', 'season_summer', 'season_winter',
                        'tag_stayed_0', 'tag_stayed_1', 'tag_stayed_2', 'tag_stayed_3', 'tag_stayed_4', 'tag_stayed_5',
                        'trip_type_Couple', 'trip_type_Solo']

# Числовые признаки
numerical_features = ['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts',
                      'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given',
                      'reviewer_score', 'lat', 'lng', 'sample', 'negative_review_sentiment', 'positive_review_sentiment']

**Выводы по разделу 3:**

В данном разделе был проведен анализ текстовых отзывов на предмет получения информации о положительном либо отрицательном оттенке отзыва




# 4. Анализ мультиколлинеарности.

4.1. Анализ мультиколлиниарности числовых признаков. Использование метода Пирсона.

In [ ]:
# Выбор числовых признаков для анализа мультиколлинеарности
numerical_features = ['additional_number_of_scoring', 'average_score', 'review_total_negative_word_counts',
                      'total_number_of_reviews', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'lat', 'lng', 'sample', 'negative_review_sentiment', 'positive_review_sentiment']

import seaborn as sns
import matplotlib.pyplot as plt

# Строим матрицу корреляции
correlation_matrix = data[numerical_features].corr()

# Создаем тепловую карту
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix for Numerical Features ')
plt.show()

Значений корреляции больше или равных 0,95 не выявлено. Сохраняем все признаки.

4.2. Анализ мультиколлинеарности категориальных  признаков. Использование метода Спирмана

In [ ]:
# Категориальные признаки
categorical_features = ['reviewer_nationality_0', 'reviewer_nationality_1', 'reviewer_nationality_2', 'reviewer_nationality_3',
                        'reviewer_nationality_4', 'reviewer_nationality_5', 'reviewer_nationality_6', 'reviewer_nationality_7',
                        'city_encoded', 'month_encoded', 'season_spring', 'season_summer', 'season_winter',
                        'tag_stayed_0', 'tag_stayed_1', 'tag_stayed_2', 'tag_stayed_3', 'tag_stayed_4', 'tag_stayed_5',
                        'trip_type_Couple', 'trip_type_Solo']



In [ ]:
# Применяем метод Спирмана для оценки корреляции между категориальными признаками
correlation_matrix_spearman = data[categorical_features].corr(method='spearman')

# Создаем тепловую карту
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix_spearman, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix for Categorical Features (Spearman)')
plt.show()

Значений корреляции больше или равных 0,95 не выявлено. Сохраняем все признаки.

**Вывод по разделу 4:**

В данном разделе была проведен анализ мультиколлиниарности числовых признаков с использованием метода Пирсона и анализ мультиколлинеарности категориальных  признаков с использованием метода Спирмана. Значений корреляции больше или равных 0,95 не выявлено. Сохраняем все признаки.


# 5. Отбор признаков на основе их важности


5.1. Тест хи-квадрат для категориальных признаков

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей)  
X = data.drop(['reviewer_score'], axis = 1)  
y = data['reviewer_score']  
y=y.astype('int')

...
from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[categorical_features], y)[0], index=categorical_features)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Количество категориальных признаков составляет 21, что, в принципе, приемлемо для обучения. Оставляем количество без изменений. 

5.2. Тест ANOVA для числовых признаков

In [ ]:
from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[numerical_features], y)[0], index = numerical_features)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Количество числовых признаков составляет 11. Признак 'sample' ничтожен по своей важности. Удаляем признак 'sample'.

In [ ]:
# Удаление признака 'sample'
#data = data.drop('sample', axis=1)
#data.info()

**Вывод по разделу 5:**

В данном разделе был проведен отбор признаков на основе их важности тестами хи-квадрат для категориальных признаков и тестом ANOVA для числовых признаков.  Был выявлен признак 'sample', ничтожный по своей важности. Данный признак был удален из датасета.


# 6. Реализация машинного алгоритма и проверка его эффективности. 

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split  
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAPE:', metrics.mean_absolute_percentage_error (y_test, y_pred))

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

predict_submission = model.predict(test_data)

sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)



**Вывод по разделу 6:**

В данном разделе был реализован машинный алгоритм и была проведена оценка его эффективности. Метрика MAPE (mean absolute percentage error) получила значение 0.11276306158480247

# 7. Общий вывод по проекту

Работа в рамках проекта позволила решить следующие задачи:
* Удаление строковых значений;
* Очистка от пропущенных значений; 
* Создание новых признаков;
* Преобразование признаков;
* Отбор признаков;

При помощи этих шагов в дальнейшем появилась возможность повысить качество предсказания и оценить данное качество, ориентируясь на метрику MAPE.